<a href="https://colab.research.google.com/github/liangyuRain/chest-xray-pneumonia/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os

import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import torch.utils.data

import matplotlib as plt


In [9]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    losses = []
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(losses)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += model.testLoss(output, label).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, test_accuracy


In [12]:
LR = 0.1
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9
EPOCHS = 100
BATCH_SIZE = 32
USE_CUDA = True
SEED = 0
LOG_INTERVAL = 100


In [4]:
use_cuda = USE_CUDA and torch.cuda.is_available()
torch.manual_seed(SEED)
device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)

import multiprocessing
print('num cpus:', multiprocessing.cpu_count())
kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}


Using device cuda
num cpus: 8


In [5]:
model = torchvision.models.resnet18(pretrained=True)

# avoid training pretrained model
for param in model.parameters():
    param.requires_grad = False

# set categories to 2
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), 
                      lr=LR, 
                      momentum=MOMENTUM, 
                      weight_decay=WEIGHT_DECAY)

model.loss = nn.CrossEntropyLoss()
model.testLoss = nn.CrossEntropyLoss(reduction='sum')


In [6]:
INPUT_SIZE = (224, 224)


class ImageLoader(object):

    def __init__(self, batchSize):
        super(ImageLoader, self).__init__()
        transform_train = transforms.Compose([
            transforms.Resize(size=INPUT_SIZE),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        transform_test = transforms.Compose([
            transforms.Resize(size=INPUT_SIZE),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        train_dataset = datasets.ImageFolder(root='chest_xray_dataset/train',
                                             transform=transform_train)
        self.trainloader = torch.utils.data.DataLoader(train_dataset,
                                                       batch_size=batchSize,
                                                       shuffle=True,
                                                       num_workers=2)

        test_dataset = datasets.ImageFolder(root='chest_xray_dataset/test',
                                            transform=transform_test)
        self.testloader = torch.utils.data.DataLoader(test_dataset,
                                                      batch_size=batchSize,
                                                      shuffle=False,
                                                      num_workers=2)

        self.classes = ('normal', 'pneumonia')


In [7]:
def plot(data, title, xlab, ylab):
    x_val = [x[0] for x in data]
    y_val = [x[1] for x in data]
    plt.plot(x_val, y_val)
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()


In [13]:
loader = ImageLoader(batchSize=BATCH_SIZE)
train_losses = []
test_losses = [] 
test_accuracies = []

try:
    for epoch in range(EPOCHS + 1):
        train_loss = train(model, device, loader.trainloader, optimizer, epoch, LOG_INTERVAL)
        test_loss, test_accuracy = test(model, device, loader.testloader)
        train_losses.append((epoch, train_loss))
        test_losses.append((epoch, test_loss))
        test_accuracies.append((epoch, test_accuracy))
except KeyboardInterrupt as ke:
    print('Interrupted')
except:
    import traceback
    traceback.print_exc()
finally:
    plot(train_losses, 'Train Losses', 'Epochs', 'Loss')
    plot(test_losses, 'Test Losses', 'Epochs', 'Loss')
    plot(test_accuracies, 'Test Accuracies', 'Epochs', 'Accuracy')


Train Epoch: 0 [0/5216 (0%)]	Loss: 2.782683


Train Epoch: 0 [3200/5216 (61%)]	Loss: 0.000000



Test set: Average loss: 39.8822, Accuracy: 539/624 (86%)



Train Epoch: 1 [0/5216 (0%)]	Loss: 3.701675


Train Epoch: 1 [3200/5216 (61%)]	Loss: 0.000000


...


Test set: Average loss: 33.3870, Accuracy: 532/624 (85%)



Train Epoch: 38 [0/5216 (0%)]	Loss: 9.125591


Train Epoch: 38 [3200/5216 (61%)]	Loss: 26.966408


Interrupted


AttributeError: module 'matplotlib' has no attribute 'plot'